### Librerias

In [1]:
import re, string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
from IPython.display import display, Markdown
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
import torch
from torch.utils.data import TensorDataset, DataLoader
from torchmetrics import F1Score
import torchmetrics
pd.set_option('display.max_colwidth', None)


c:\Users\gfern\Documentos\A_Data_Camp\Practicas_Proy\.env_NLP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### DataSet Kaggle. Analisis Sentimientos

Voy a realizar estudio de sentimientos con este dataset, la idea es ver el accuracy usando un modelo preentrenado. https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news

In [2]:
data = pd.read_csv('all-data.csv', names=['sentiment','coment'],  encoding='ISO-8859-1')

In [42]:
data.head()


,sentiment,coment,text_cleaned
0,neutral,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",according to gran the company has no plans to move all production to russia although that is where the company is growing
1,neutral,"Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .",technopolis plans to develop in stages an area of no less than 100000 square meters in order to host companies working in computer technologies and telecommunications the statement said
2,negative,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .",the international electronic industry company elcoteq has laid off tens of employees from its tallinn facility contrary to earlier layoffs the company contracted the ranks of its office workers the daily postimees reported
3,positive,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,with the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability
4,positive,"According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .",according to the company updated strategy for the years 20092012 basware targets longterm net sales growth in the range of 20 40 with an operating profit margin of 10 20 of net sales


In [4]:
data['sentiment'].value_counts()

sentiment
neutral     2879
positive    1363
negative     604
Name: count, dtype: int64

### Preprocesando el texto

In [5]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
# Reemplazando Forward - Slash Character
data['text_cleaned']=data['coment'].str.replace(r"\/",'')

# Me aseguro que todos los datos son tipo string
data['text_cleaned']=data['text_cleaned'].astype(str)

# Remove Punctuation
translator = str.maketrans('', '', string.punctuation)
data['text_cleaned'] = data['text_cleaned'].str.translate(translator)

# Remove digits
data['text_cleaned']=data['text_cleaned'].str.replace(r"\d+", "")

# Remove single letters
data['text_cleaned'] = data['text_cleaned'].str.replace(r"\b[a-zA-Z]\b", "", regex=True)

# Remove running space
data['text_cleaned']=data['text_cleaned'].str.replace(r"\s{2,}+", "")

# Make the text lowercase
data['text_cleaned']=data['text_cleaned'].str.lower()


In [7]:
data.head()

,sentiment,coment,text_cleaned
0,neutral,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",according to gran the company has no plans to move all production to russia although that is where the company is growing
1,neutral,"Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .",technopolis plans to develop in stages an area of no less than 100000 square meters in order to host companies working in computer technologies and telecommunications the statement said
2,negative,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .",the international electronic industry company elcoteq has laid off tens of employees from its tallinn facility contrary to earlier layoffs the company contracted the ranks of its office workers the daily postimees reported
3,positive,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,with the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability
4,positive,"According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .",according to the company updated strategy for the years 20092012 basware targets longterm net sales growth in the range of 20 40 with an operating profit margin of 10 20 of net sales


In [8]:
X=data.iloc[:,2]
X.head()

0                                                                                                         according to gran  the company has no plans to move all production to russia  although that is where the company is growing 
1                                          technopolis plans to develop in stages an area of no less than 100000 square meters in order to host companies working in computer technologies and telecommunications  the statement said 
2    the international electronic industry company elcoteq has laid off tens of employees from its tallinn facility  contrary to earlier layoffs the company contracted the ranks of its office workers  the daily postimees reported 
3                        with the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability 
4                                       according to the company  updated st

### Preparando y Dividiendo conjunto de entrenamiento y test

In [9]:
y=data.iloc[:,0]

In [10]:
ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [0])],   
    remainder='passthrough')
data_tf = ct.fit_transform(data)

In [11]:
data_tf=pd.DataFrame(data_tf, columns=['negative','neutral','positive','text','text_cleaned'])

In [12]:
data_tf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   negative      4846 non-null   object
 1   neutral       4846 non-null   object
 2   positive      4846 non-null   object
 3   text          4846 non-null   object
 4   text_cleaned  4846 non-null   object
dtypes: object(5)
memory usage: 189.4+ KB


In [13]:
data_tf.head()

,negative,neutral,positive,text,text_cleaned
0,0.0,1.0,0.0,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",according to gran the company has no plans to move all production to russia although that is where the company is growing
1,0.0,1.0,0.0,"Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .",technopolis plans to develop in stages an area of no less than 100000 square meters in order to host companies working in computer technologies and telecommunications the statement said
2,1.0,0.0,0.0,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .",the international electronic industry company elcoteq has laid off tens of employees from its tallinn facility contrary to earlier layoffs the company contracted the ranks of its office workers the daily postimees reported
3,0.0,0.0,1.0,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,with the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability
4,0.0,0.0,1.0,"According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .",according to the company updated strategy for the years 20092012 basware targets longterm net sales growth in the range of 20 40 with an operating profit margin of 10 20 of net sales


In [14]:
X=data_tf.iloc[:,4]
X=pd.DataFrame(X)

In [15]:
y=data_tf.iloc[:,0:3]
y=pd.DataFrame(y)

In [16]:
y.head()

,negative,neutral,positive
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0


In [17]:
X.head()

,text_cleaned
0,according to gran the company has no plans to move all production to russia although that is where the company is growing
1,technopolis plans to develop in stages an area of no less than 100000 square meters in order to host companies working in computer technologies and telecommunications the statement said
2,the international electronic industry company elcoteq has laid off tens of employees from its tallinn facility contrary to earlier layoffs the company contracted the ranks of its office workers the daily postimees reported
3,with the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability
4,according to the company updated strategy for the years 20092012 basware targets longterm net sales growth in the range of 20 40 with an operating profit margin of 10 20 of net sales


In [18]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
X_train=pd.DataFrame(X_train)
X_test=pd.DataFrame(X_test)
y_train=pd.DataFrame(y_train)
y_test=pd.DataFrame(y_test)

In [20]:
y_train.head()

,negative,neutral,positive
432,0.0,1.0,0.0
3071,0.0,1.0,0.0
289,0.0,0.0,1.0
2588,0.0,1.0,0.0
4121,0.0,1.0,0.0


In [21]:
train=X_train.join(y_train)
test=X_test.join(y_test)

In [22]:
train.head(10)

,text_cleaned,negative,neutral,positive
432,in russia raisio food division home market stretches all the way to vladivostok,0.0,1.0,0.0
3071,operators only need to learn how to use one device for multiple levels of applications including voicedirected operations,0.0,1.0,0.0
289,the company expects its net sales in the first half of 2008 to reach the previous year level,0.0,0.0,1.0
2588,the bridge will be 12 km long and is located between anasmotet by the road e20 and the new traffic junction in marieholm by the road e45,0.0,1.0,0.0
4121,nokia and capcom announced that resident evil degeneration will be released on ngage later this year,0.0,1.0,0.0
4489,at the same time am delighted by the fact that we were able to accomplish the organization with our own team members,0.0,0.0,1.0
535,with the launch of new 3g handsets nokia aims to become the winner in china 3g market as it did in the 2g market,0.0,0.0,1.0
1479,the chf is great product,0.0,0.0,1.0
188,eps from continuing operations came in at 030 eur up from 017,0.0,0.0,1.0
3573,earnings per share eps in the first quarter amounted to loss of eur0 20,0.0,1.0,0.0


In [23]:
test.head()

,text_cleaned,negative,neutral,positive
3207,the company was supposed to deliver machinery to veneer mill in the tomsk region in russia,0.0,1.0,0.0
1684,unc charlotte would also deploy ssh tectia connector to enable secure application connectivity,0.0,1.0,0.0
1044,in 2009 lee man had combined annual production capacity of close to 45 million tonnes of paper and 300000 tonnes of pulp,0.0,1.0,0.0
4145,that very high figure on the european scale noop said recalling however that this also includes beer bought by finnish tourists,0.0,1.0,0.0
1538,in finland the corresponding service is alma media etuovicom finland most popular and best known nationwide online service for home and property sales,0.0,1.0,0.0


In [24]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

### Ejecucion del Modelo

In [25]:
train.columns

Index(['text_cleaned', 'negative', 'neutral', 'positive'], dtype='object')

In [26]:
# Cargar los archivos de entrenamiento y prueba
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Crear un tokenizer para el modelo
tokenizer = AutoTokenizer.from_pretrained('SamLowe/roberta-base-go_emotions')

# Preparar los datos para el modelo
train_encodings = tokenizer(train_df['text_cleaned'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_df['text_cleaned'].tolist(), truncation=True, padding=True)

# Convertir las etiquetas de destino a one-hot encoding
train_labels = train_df[['negative', 'neutral', 'positive']].values
test_labels = test_df[['negative', 'neutral', 'positive']].values

# Crear un modelo de Hugging Face para análisis de sentimiento
model = AutoModelForSequenceClassification.from_pretrained('SamLowe/roberta-base-go_emotions', num_labels=3,
                                                           ignore_mismatched_sizes=True)

# Crear un conjunto de datos de entrenamiento
train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels)
)

# Crear un conjunto de datos de prueba
test_dataset = TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels)
)

# Crear un cargador de datos de entrenamiento
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Crear un cargador de datos de prueba
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Entrenar el modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

for epoch in range(5):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = criterion(outputs.logits, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}')

model.eval()

c:\Users\gfern\Documentos\A_Data_Camp\Practicas_Proy\.env_NLP\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at SamLowe/roberta-base-go_emotions and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([28]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([28, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 0.5028824640568269
Epoch 2, Loss: 0.28886513402465147
Epoch 3, Loss: 0.2074594546303224
Epoch 4, Loss: 0.1504183866720433
Epoch 5, Loss: 0.1011270933111204


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

### Guardando el Modelo

In [27]:
model.save_pretrained('sentiment_model_finance_twitter')

### Cargando el Modelo

In [31]:
# Cargar el modelo y el tokenizer
#modelo = AutoModel.from_pretrained('C:/Users/gfern/Documentos/A_Data_Camp/Practicas_Proy/sentiment_model_finance_twitter')
#tokenizer = AutoTokenizer.from_pretrained('C:/Users/gfern/Documentos/A_Data_Camp/Practicas_Proy/sentiment_model_finance_twitter')

Some weights of RobertaModel were not initialized from the model checkpoint at C:/Users/gfern/Documentos/A_Data_Camp/Practicas_Proy/sentiment_model_finance_twitter and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Predicciones con el modelo

In [32]:
# Realizar predicciones en el conjunto de prueba
model.eval() 
predicciones = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, predicted = torch.max(logits, dim=1)
        predicciones.extend(predicted.cpu().numpy())


# Calcular la accuracy forma teorica en la siguiente linea se aplica con la libreria torchmetrics

test_labels = test_df[['negative', 'neutral', 'positive']].values.argmax(axis=1)
accuracy = (predicciones == test_labels).sum() / len(test_labels)
print(f'Accuracy: {accuracy:.4f}')



Accuracy: 0.8577


### Metricas de Rendimiento del Modelo

In [33]:
y_pred=[int(x) for x in predicciones]

In [36]:
# Convertir las predicciones y etiquetas a tensores de PyTorch
preds = torch.tensor(predicciones)
target = torch.tensor(test_labels)

# Calcular la accuracy utilizando torchmetrics
acc = torchmetrics.functional.classification.multiclass_accuracy(
    preds, target, num_classes=3)

print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8535


In [35]:
# Calcular el F1 score
f1 = F1Score(num_classes=3, average='macro', task='multiclass')
f1_score = f1(preds, target)
print(f'F1 Score: {f1_score:.4f}')

F1 Score: 0.8541


In [37]:
# Generate the classification report de Sklearn aplicado sobre las listas test_labels y predicciones
report= classification_report(test_labels, predicciones, output_dict=False)

# Print the report
print(report)

              precision    recall  f1-score   support

           0       0.90      0.85      0.87       110
           1       0.90      0.87      0.88       571
           2       0.78      0.84      0.81       289

    accuracy                           0.86       970
   macro avg       0.86      0.85      0.85       970
weighted avg       0.86      0.86      0.86       970

